In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
import pandas as pd
import numpy as np
import streamlit as st
import torch

c:\Users\duyngocadn\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.is_available()

True

In [4]:
%%writefile app.py

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
import pandas as pd
import numpy as np
import streamlit as st
import torch
import requests
from bs4 import BeautifulSoup
import os
import gc
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import time

st.set_page_config(layout = "wide")
st.set_option('deprecation.showPyplotGlobalUse', False)

def CrawData_TuoiTre(url):
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")
        
        # get title
    try: 
        title = soup.find("h1", class_="article-title").text.strip()
    except:
        print(os.error("Không tìm thấy"))
                    
        # get abstract
    try:
        abstract = soup.find("h2", class_="sapo").text
        if abstract.startswith("TTO - ") == True:
            abstract = abstract.replace("TTO - ", "", 1)
    except:
        print(os.error("Không tìm thấy"))
        
        # get body
    body = soup.find("div", id="main-detail-body")
    content = ""
    for i in body.findChildren("p", recursive=False):
        content += str(i.text.strip() + " ")
            
        # append data
    data = ({
        "title": title,
            "abstract": abstract.strip(),
            "content": content.strip(),
        })

    return data


def CrawData_ThanhNien(url):
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")
        
        # get title
    try: 
        title = soup.find("h1", class_="details__headline cms-title").text.strip()
    except:
        print(os.error("Không tìm thấy"))
                    
        # get abstract
    try:
        abstract = soup.find("div", class_="sapo cms-desc").text
    except:
        print(os.error("Không tìm thấy"))
        
        # get body
    body = soup.find("div", class_= "cms-body detail")
    content = ""
    for i in body.findChildren("p", recursive=False):
        content += str(i.text.strip() + " ")
            
        # append data
    data = ({
        "title": title,
            "abstract": abstract.strip(),
            "content": content.strip(),
        })

    return data
    
def CrawData_VNexpress(url):
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")
        
        # get title
    try: 
        title = soup.find("h1", class_="title-detail").text.strip()
    except:
        print(os.error("Không tìm thấy"))
                    
        # get abstract
    try:
        abstract = soup.find("p", class_="description").text
    except:
        print(os.error("Không tìm thấy"))
        
        # get body
    body = soup.find("article", class_="fck_detail")
    content = ""
    for i in body.findChildren("p", class_ = "Normal", recursive=False):
        content += str(i.text.strip() + " ")
            
        # append data
    data = ({
        "title": title,
            "abstract": abstract.strip(),
            "content": content.strip(),
        })

    return data


def CrawData_VTCNews(url):
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")
        
        # get title
    try: 
        title = soup.find("h1", class_="font28 bold lh-1-3").text.strip()
    except:
        print(os.error("Không tìm thấy"))
                    
        # get abstract
    try:
        abstract = soup.find("h2", class_="font18 bold inline-nb").text.strip()
    except:
        print(os.error("Không tìm thấy"))
        
        # get body
    body = soup.find("div", class_="edittor-content box-cont mt15 clearfix")
    content = ""
    try:
        for i in body.findChildren("p", recursive=False):
            content += str(i.text.strip() + " ")
    except:
        print(os.error("Không tìm thấy"))
        # append data
    data = ({
        "title": title,
            "abstract": abstract.strip(),
            "content": content.strip(),
        })

    return data


def CrawData_VTV(url):
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")
        
        # get title
    try: 
        title = soup.find("h1", class_="title_detail").text.strip()
    except:
        print(os.error("Không tìm thấy"))
                    
        # get abstract
    try:
        abstract = soup.find("h2", class_="sapo").text
        if abstract.startswith("VTV.vn - ") == True:
            abstract = abstract.replace("VTV.vn - ", "", 1)
    except:
        print(os.error("Không tìm thấy"))
        
        # get body
    body = soup.find("div", id="entry-body")
    content = ""
    for i in body.findChildren("p", recursive=False):
        content += str(i.text.strip() + " ")
            
        # append data
    data = ({
        "title": title,
            "abstract": abstract.strip(),
            "content": content.strip(),
        })

    return data


def CrawData_DanTri(url):
    news = requests.get(url)
    soup = BeautifulSoup(news.content, "html.parser")
        
        # get title
    try: 
        title = soup.find("h1", class_="title-page detail").text.strip()
    except:
        print(os.error("Không tìm thấy"))
                    
        # get abstract
    try:
        abstract = soup.find(class_="singular-sapo").text
        if abstract.startswith("(Dân trí) - ") == True:
            abstract = abstract.replace("(Dân trí) - ", "", 1)
    except:
        print(os.error("Không tìm thấy"))
        
        # get body
    body = soup.find("div", class_="singular-content")
    content = ""
    for i in body.findChildren("p", recursive=False):
        content += str(i.text.strip() + " ")
            
        # append data
    data = ({
        "title": title,
            "abstract": abstract.strip(),
            "content": content.strip(),
        })

    return data

try: 
    st.title('ABSTRACT TEXT SUMMARIZATION FOR DIGITAL NEWSPAPER')

    with st.sidebar:
        st.header("Các trang báo tham khảo")
        st.write(
            """
            - Tuổi trẻ: https://tuoitre.vn/
            - VNexpress: https://vnexpress.net/
            - Thanh niên: https://thanhnien.vn/
            - VTC News: https://vtc.vn/
            - VTV News: https://vtv.vn/
            - Dân trí: https://dantri.com.vn/
            """
        )
    col1, col2 = st.columns([1, 3])
    line = ""
    content = "Mời bạn nhập bài báo"
    with col1:
        # Chọn model
        model_name = st.selectbox(
        '**Chọn model**',
        ('None', 'ViT5-Base', 'ViT5-Large', 'BARTPho'))

        # Chọn trang báo
        web = st.selectbox('**Chọn trang báo**', ('None', 'Thanh niên', 'Tuổi trẻ', 'VNexpress', 'VTC News', "VTV News", "Dân trí"))
        if web == 'None':
            link_web = ""
        else:
            link_web = st.text_input('Link bài báo')
        summary_check = st.button("Summary")


    with col2: 
        st.subheader("Bài báo")
        
        if summary_check:
            if web == 'None':
                content = "Mời bạn nhập bài báo"
            else:
                if web == "Thanh niên":
                    content = CrawData_ThanhNien(link_web)["content"]
                elif web == "Tuổi trẻ":
                    content = CrawData_TuoiTre(link_web)["content"]
                elif web == "VNexpress":
                    content = CrawData_VNexpress(link_web)["content"]
                elif web == "VTC News":
                    content = CrawData_VTCNews(link_web)["content"]  
                elif web == "VTV News": 
                    content = CrawData_VTV(link_web)["content"]  
                elif web == "Dân trí":
                    content = CrawData_DanTri(link_web)["content"]  
        sentence= st.text_area(label = '', value=content, height= 300, label_visibility="collapsed", key="news_content")    
    if summary_check:
        with col1:
            with st.spinner('Wait for it...'):
                if model_name == "None":
                    st.error("Chọn model")
                else:
                    if model_name == "ViT5-Base":
                        tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
                        model = AutoModelForSeq2SeqLM.from_pretrained(r"D:\\Đại học\\Năm 3\\Kỳ 1\\Xử lí ngôn ngữ tự nhiên\\Đồ án\Demo\\Model\\ViT5-Base")
                    elif model_name == "ViT5-Large":
                        tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-large")  
                        model = AutoModelForSeq2SeqLM.from_pretrained(r"D:\\Đại học\\Năm 3\\Kỳ 1\\Xử lí ngôn ngữ tự nhiên\\Đồ án\\Demo\\Model\\ViT5-Large")
                    elif model_name == "BARTPho":
                        tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")  
                        model = AutoModelForSeq2SeqLM.from_pretrained(r"D:\\Đại học\\Năm 3\\Kỳ 1\\Xử lí ngôn ngữ tự nhiên\\Đồ án\\Demo\\Model\\BARTPho")
                    model.to('cuda')  

    if summary_check:

        text =  sentence + " </s>"
        encoding = tokenizer(text, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
        outputs = model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=64,
            early_stopping=True)
        for output in outputs:
            line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            del model
            del tokenizer, sentence, outputs

    with col2:
        st.subheader("Tóm tắt")
        st.success(body=line)

except:
    e = RuntimeError('Tràn bộ nhớ')
    st.exception(e)


gc.collect()
torch.cuda.empty_cache()

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()
    torch.cuda.empty_cache()
free_gpu_cache()  

Overwriting app.py


In [5]:
streamlit run "D:\Đại học\Năm 3\Kỳ 1\Xử lí ngôn ngữ tự nhiên\Đồ án\Demo\app.py"

In [27]:
import gc
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
gc.collect()
torch.cuda.empty_cache()

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()
    torch.cuda.empty_cache()
free_gpu_cache() 

2022-12-18 21:32:52.574 INFO    numba.cuda.cudadrv.driver: reset context of device 0


Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 64% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  4% | 66% |


In [5]:
!ngrok authtoken 2F80l9KYKzY6BiWxZLOF7WWPPIv_4SY2Qa71NHfn9YyqfDy1A

Authtoken saved to configuration file: C:\Users\duyngocadn/.ngrok2/ngrok.yml


In [4]:
from pyngrok import ngrok
import os
get_ipython().system = os.system
!taskkill /f /im ngrok.exe
public_url = ngrok.connect(8501)
print(f"Open this link in a new tab to interact with the demo app: {public_url}")

2022-12-25 20:01:31.245 INFO    pyngrok.process: ngrok process starting: 27256
2022-12-25 20:01:31.311 INFO    pyngrok.process: t=2022-12-25T20:01:31+0700 lvl=info msg="no configuration paths supplied"

2022-12-25 20:01:31.312 INFO    pyngrok.process: t=2022-12-25T20:01:31+0700 lvl=info msg="using configuration at default config path" path=C:\\Users\\duyngocadn/.ngrok2/ngrok.yml

2022-12-25 20:01:31.313 INFO    pyngrok.process: t=2022-12-25T20:01:31+0700 lvl=info msg="open config file" path=C:\\Users\\duyngocadn\\.ngrok2\\ngrok.yml err=nil

2022-12-25 20:01:31.315 INFO    pyngrok.process: t=2022-12-25T20:01:31+0700 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2022-12-25 20:01:32.206 INFO    pyngrok.process: t=2022-12-25T20:01:32+0700 lvl=info msg="tunnel session started" obj=tunnels.session

2022-12-25 20:01:32.207 INFO    pyngrok.process: t=2022-12-25T20:01:32+0700 lvl=info msg="client session established" obj=csess id=0dcd2f532092

2022-12-25 20:01:32.219 INFO    

Open this link in a new tab to interact with the demo app: http://f242-118-69-89-19.ngrok.io


2022-12-25 20:01:32.827 INFO    pyngrok.process: t=2022-12-25T20:01:32+0700 lvl=info msg="started tunnel" obj=tunnels name=http-8501-a7e273f0-f3bb-4984-89a4-99feafe5e4e0 addr=http://localhost:8501 url=https://f242-118-69-89-19.ngrok.io

2022-12-25 20:01:32.827 INFO    pyngrok.process: t=2022-12-25T20:01:32+0700 lvl=info msg=end pg=/api/tunnels id=6adcd3e752bb5f65 status=201 dur=579.2867ms



2022-12-25 20:01:45.213 WARNING pyngrok.process: t=2022-12-25T20:01:45+0700 lvl=warn msg="failed to open private leg" id=6f426a2759d1 privaddr=localhost:8501 err="dial tcp [::1]:8501: connectex: No connection could be made because the target machine actively refused it."

2022-12-25 20:01:49.902 WARNING pyngrok.process: t=2022-12-25T20:01:49+0700 lvl=warn msg="failed to open private leg" id=a62928caea4e privaddr=localhost:8501 err="dial tcp [::1]:8501: connectex: No connection could be made because the target machine actively refused it."

2022-12-25 20:01:53.555 WARNING pyngrok.process: t=2022-12-25T20:01:53+0700 lvl=warn msg="failed to open private leg" id=1481175b4c65 privaddr=localhost:8501 err="dial tcp [::1]:8501: connectex: No connection could be made because the target machine actively refused it."

